**Prepare MIMICII benchmark data**

Based on https://github.com/YerevaNN/mimic3-benchmarks

In [1]:
NB_DIR = %pwd

In [2]:
RAW_DATA = '/data1/MIMIC-III/RAW/'
INTERIM_DATA = f'{RAW_DATA}/../interim/'
PROCESSED_DATA = f'{RAW_DATA}/../processed/'

In [3]:
%mkdir -p $INTERIM_DATA

In [4]:
%mkdir -p $PROCESSED_DATA

In [5]:
MIMIC3_BENCHMARK_LOCATION = f'{NB_DIR}/../mimic3-benchmarks/'

# Imports

In [6]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [7]:
import yaml

In [8]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [9]:
from pandas import DataFrame, Series

# Extract subjects

Based on https://github.com/YerevaNN/mimic3-benchmarks/blob/master/scripts/extract_subjects.py

## Read in data

In [10]:
def dataframe_from_csv(path, header=0, index_col=0):
    return pd.read_csv(path, header=header, index_col=index_col)

In [11]:
def read_patients_table(mimic3_path):
    pats = pd.read_csv(os.path.join(mimic3_path, 'PATIENTS.csv'), header=0, index_col=0)
    pats = pats[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']]
    pats.DOB = pd.to_datetime(pats.DOB)
    pats.DOD = pd.to_datetime(pats.DOD)
    return pats

def read_admissions_table(mimic3_path):
    admits = pd.read_csv(os.path.join(mimic3_path, 'ADMISSIONS.csv'), header=0, index_col=0)
    admits = admits[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'DIAGNOSIS']]
    admits.ADMITTIME = pd.to_datetime(admits.ADMITTIME)
    admits.DISCHTIME = pd.to_datetime(admits.DISCHTIME)
    admits.DEATHTIME = pd.to_datetime(admits.DEATHTIME)
    return admits

def read_icustays_table(mimic3_path):
    stays = pd.read_csv(os.path.join(mimic3_path, 'ICUSTAYS.csv'), header=0, index_col=0)
    stays.INTIME = pd.to_datetime(stays.INTIME)
    stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
    return stays

In [12]:
patients = read_patients_table(RAW_DATA)
admits = read_admissions_table(RAW_DATA)
stays = read_icustays_table(RAW_DATA)

In [13]:
patients.head()

,SUBJECT_ID,GENDER,DOB,DOD
ROW_ID,,,,
234,249,F,2075-03-13,NaT
235,250,F,2164-12-27,2188-11-22
236,251,M,2090-03-15,NaT
237,252,M,2078-03-06,NaT
238,253,F,2089-11-26,NaT


In [14]:
admits.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS
ROW_ID,,,,,,,
21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,WHITE,BENZODIAZEPINE OVERDOSE
22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,WHITE,BRAIN MASS
24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,WHITE,INTERIOR MYOCARDIAL INFARCTION
25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,WHITE,ACUTE CORONARY SYNDROME


In [15]:
stays.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
ROW_ID,,,,,,,,,,,
365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [16]:
stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61532 entries, 365 to 59810
Data columns (total 11 columns):
SUBJECT_ID        61532 non-null int64
HADM_ID           61532 non-null int64
ICUSTAY_ID        61532 non-null int64
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null int64
LAST_WARDID       61532 non-null int64
INTIME            61532 non-null datetime64[ns]
OUTTIME           61522 non-null datetime64[ns]
LOS               61522 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 5.6+ MB


In [17]:
DataFrameSummary(stays).summary()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
count,61532,61532,61532,NaN,NaN,NaN,61532,61532,NaN,NaN,61522
mean,33888.6,149954,249963,NaN,NaN,NaN,33.0857,32.9782,NaN,NaN,4.91797
std,28127.7,28898.9,28890.6,NaN,NaN,NaN,19.1022,19.0286,NaN,NaN,9.63878
min,2,100001,200001,NaN,NaN,NaN,7,7,NaN,NaN,0.0001
25%,12047.5,124934,224936,NaN,NaN,NaN,14,14,NaN,NaN,1.10803
50%,24280.5,149912,249940,NaN,NaN,NaN,33,33,NaN,NaN,2.09225
75%,54191.5,174997,274972,NaN,NaN,NaN,52,52,NaN,NaN,4.48318
max,99999,199999,299999,NaN,NaN,NaN,57,57,NaN,NaN,173.072
counts,61532,61532,61532,61532,61532,61532,61532,61532,61532,61522,61522
uniques,46476,57786,61532,3,6,6,16,17,61531,61518,40175


## Remove duplicates because of ICU transfers

**ADD EXPLANATION:**

In [18]:
def remove_icustays_with_transfers(stays):
    stays = stays.loc[(stays.FIRST_WARDID == stays.LAST_WARDID) & (stays.FIRST_CAREUNIT == stays.LAST_CAREUNIT)]
    return stays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE', 'INTIME', 'OUTTIME', 'LOS']]

In [19]:
stays_noTransf = remove_icustays_with_transfers(stays)

In [20]:
stays_noTransf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55830 entries, 365 to 59810
Data columns (total 8 columns):
SUBJECT_ID       55830 non-null int64
HADM_ID          55830 non-null int64
ICUSTAY_ID       55830 non-null int64
LAST_CAREUNIT    55830 non-null object
DBSOURCE         55830 non-null object
INTIME           55830 non-null datetime64[ns]
OUTTIME          55820 non-null datetime64[ns]
LOS              55820 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(3), object(2)
memory usage: 3.8+ MB


In [21]:
stays = stays_noTransf

In [22]:
stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55830 entries, 365 to 59810
Data columns (total 8 columns):
SUBJECT_ID       55830 non-null int64
HADM_ID          55830 non-null int64
ICUSTAY_ID       55830 non-null int64
LAST_CAREUNIT    55830 non-null object
DBSOURCE         55830 non-null object
INTIME           55830 non-null datetime64[ns]
OUTTIME          55820 non-null datetime64[ns]
LOS              55820 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(3), object(2)
memory usage: 3.8+ MB


## Merge tables

Merge `stays` with `admits` and `patients`

In [23]:
def merge_on_subject_admission(table1, table2):
    return table1.merge(table2, how='inner', left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])

def merge_on_subject(table1, table2):
    return table1.merge(table2, how='inner', left_on=['SUBJECT_ID'], right_on=['SUBJECT_ID'])

In [24]:
stays = merge_on_subject_admission(stays, admits)

In [25]:
stays.head(2)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LAST_CAREUNIT,DBSOURCE,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS
0,268,110404,280836,MICU,carevue,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,HISPANIC OR LATINO,DYSPNEA
1,269,106296,206613,MICU,carevue,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,2170-11-05 11:04:00,2170-11-27 18:00:00,NaT,WHITE,SEPSIS;PILONIDAL ABSCESS


In [26]:
stays = merge_on_subject(stays, patients)

In [27]:
stays.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LAST_CAREUNIT,DBSOURCE,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD
0,268,110404,280836,MICU,carevue,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,HISPANIC OR LATINO,DYSPNEA,F,2132-02-21,2198-02-18
1,269,106296,206613,MICU,carevue,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,2170-11-05 11:04:00,2170-11-27 18:00:00,NaT,WHITE,SEPSIS;PILONIDAL ABSCESS,M,2130-09-30,NaT
2,270,188028,220345,CCU,carevue,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939,2128-06-23 18:26:00,2128-06-27 12:31:00,NaT,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,M,2048-05-26,NaT
3,272,164716,210407,CCU,carevue,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202,2186-12-25 21:06:00,2187-01-02 14:57:00,NaT,WHITE,PULMONARY EMBOLIS,M,2119-11-21,NaT
4,273,158689,241507,MICU,carevue,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862,2141-04-19 06:11:00,2141-04-20 17:00:00,NaT,BLACK/AFRICAN AMERICAN,POLYSUBSTANCE OVERDOSE,M,2107-08-10,NaT


### ...

## Remove duplicate stays:

In [28]:
print('REMOVE MULTIPLE STAYS PER ADMIT:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0],
          stays.SUBJECT_ID.unique().shape[0])

REMOVE MULTIPLE STAYS PER ADMIT: 55830 52834 43277


In [29]:
def filter_admissions_on_nb_icustays(stays, min_nb_stays=1, max_nb_stays=1):
    to_keep = stays.groupby('HADM_ID').count()[['ICUSTAY_ID']].reset_index()
    to_keep = to_keep.loc[(to_keep.ICUSTAY_ID>=min_nb_stays)&(to_keep.ICUSTAY_ID<=max_nb_stays)][['HADM_ID']]
    stays = stays.merge(to_keep, how='inner', left_on='HADM_ID', right_on='HADM_ID')
    return stays

In [30]:
stays = filter_admissions_on_nb_icustays(stays)

In [31]:
stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50186 entries, 0 to 50185
Data columns (total 16 columns):
SUBJECT_ID       50186 non-null int64
HADM_ID          50186 non-null int64
ICUSTAY_ID       50186 non-null int64
LAST_CAREUNIT    50186 non-null object
DBSOURCE         50186 non-null object
INTIME           50186 non-null datetime64[ns]
OUTTIME          50176 non-null datetime64[ns]
LOS              50176 non-null float64
ADMITTIME        50186 non-null datetime64[ns]
DISCHTIME        50186 non-null datetime64[ns]
DEATHTIME        4596 non-null datetime64[ns]
ETHNICITY        50186 non-null object
DIAGNOSIS        50185 non-null object
GENDER           50186 non-null object
DOB              50186 non-null datetime64[ns]
DOD              18284 non-null datetime64[ns]
dtypes: datetime64[ns](7), float64(1), int64(3), object(5)
memory usage: 6.5+ MB


In [32]:
print('REMOVED MULTIPLE STAYS PER ADMIT:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0],
          stays.SUBJECT_ID.unique().shape[0])

REMOVED MULTIPLE STAYS PER ADMIT: 50186 50186 41587


## Add age and mortality to stays

In [33]:
def add_age_to_icustays(stays):
    stays['AGE'] = (stays.INTIME - stays.DOB).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60/24/365
    stays.loc[stays.AGE<0,'AGE'] = 90
    return stays

In [34]:
stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD'],
      dtype='object')

In [35]:
stays = add_age_to_icustays(stays)

In [36]:
stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD', 'AGE'],
      dtype='object')

In [37]:
def add_inunit_mortality_to_icustays(stays):
    mortality = stays.DOD.notnull() & ((stays.INTIME <= stays.DOD) & (stays.OUTTIME >= stays.DOD))
    mortality = mortality | (stays.DEATHTIME.notnull() & ((stays.INTIME <= stays.DEATHTIME) & (stays.OUTTIME >= stays.DEATHTIME)))
    stays['MORTALITY_INUNIT'] = mortality.astype(int)
    return stays

In [38]:
stays = add_inunit_mortality_to_icustays(stays)

In [39]:
stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD', 'AGE',
       'MORTALITY_INUNIT'],
      dtype='object')

In [40]:
def add_inhospital_mortality_to_icustays(stays):
    mortality = stays.DOD.notnull() & ((stays.ADMITTIME <= stays.DOD) & (stays.DISCHTIME >= stays.DOD))
    mortality = mortality | (stays.DEATHTIME.notnull() & ((stays.ADMITTIME <= stays.DEATHTIME) & (stays.DISCHTIME >= stays.DEATHTIME)))
    stays['MORTALITY'] = mortality.astype(int)
    stays['MORTALITY_INHOSPITAL'] = stays['MORTALITY']
    return stays

In [41]:
stays = add_inhospital_mortality_to_icustays(stays)

In [42]:
stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD', 'AGE',
       'MORTALITY_INUNIT', 'MORTALITY', 'MORTALITY_INHOSPITAL'],
      dtype='object')

In [43]:
stays.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LAST_CAREUNIT,DBSOURCE,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,AGE,MORTALITY_INUNIT,MORTALITY,MORTALITY_INHOSPITAL
0,268,110404,280836,MICU,carevue,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,HISPANIC OR LATINO,DYSPNEA,F,2132-02-21,2198-02-18,66.030075,1,1,1
1,269,106296,206613,MICU,carevue,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,2170-11-05 11:04:00,2170-11-27 18:00:00,NaT,WHITE,SEPSIS;PILONIDAL ABSCESS,M,2130-09-30,NaT,40.127294,0,0,0
2,270,188028,220345,CCU,carevue,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939,2128-06-23 18:26:00,2128-06-27 12:31:00,NaT,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,M,2048-05-26,NaT,80.133229,0,0,0
3,272,164716,210407,CCU,carevue,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202,2186-12-25 21:06:00,2187-01-02 14:57:00,NaT,WHITE,PULMONARY EMBOLIS,M,2119-11-21,NaT,67.142139,0,0,0
4,273,158689,241507,MICU,carevue,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862,2141-04-19 06:11:00,2141-04-20 17:00:00,NaT,BLACK/AFRICAN AMERICAN,POLYSUBSTANCE OVERDOSE,M,2107-08-10,NaT,33.715776,0,0,0


## Remove youths (age < 18):

In [44]:
print('REMOVE PATIENTS AGE < 18:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0],
          stays.SUBJECT_ID.unique().shape[0])

REMOVE PATIENTS AGE < 18: 50186 50186 41587


In [45]:
def filter_icustays_on_age(stays, min_age=18, max_age=np.inf):
    stays = stays.loc[(stays.AGE>=min_age)&(stays.AGE<=max_age)]
    return stays

In [46]:
stays = filter_icustays_on_age(stays)

In [47]:
print('REMOVE PATIENTS AGE < 18:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0],
          stays.SUBJECT_ID.unique().shape[0])

REMOVE PATIENTS AGE < 18: 42276 42276 33798


## Save stays df

In [48]:
stays.to_csv(os.path.join(INTERIM_DATA, 'all_stays.csv'), index=False)

In [49]:
#!ls $INTERIM_DATA

In [50]:
#!du -h $INTERIM_DATA

## Create and save diagnosis df

In [51]:
def read_icd_diagnoses_table(mimic3_path):
    codes = dataframe_from_csv(os.path.join(mimic3_path, 'D_ICD_DIAGNOSES.csv'))
    codes = codes[['ICD9_CODE','SHORT_TITLE','LONG_TITLE']]
    diagnoses = dataframe_from_csv(os.path.join(mimic3_path, 'DIAGNOSES_ICD.csv'))
    diagnoses = diagnoses.merge(codes, how='inner', left_on='ICD9_CODE', right_on='ICD9_CODE')
    diagnoses[['SUBJECT_ID','HADM_ID','SEQ_NUM']] = diagnoses[['SUBJECT_ID','HADM_ID','SEQ_NUM']].astype(int)
    return diagnoses

In [52]:
diagnoses = read_icd_diagnoses_table(RAW_DATA)

In [53]:
diagnoses.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,109,172335,1,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
1,109,173633,1,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
2,109,131345,1,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
3,109,131376,1,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
4,109,135923,1,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."


In [54]:
diagnoses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634709 entries, 0 to 634708
Data columns (total 6 columns):
SUBJECT_ID     634709 non-null int64
HADM_ID        634709 non-null int64
SEQ_NUM        634709 non-null int64
ICD9_CODE      634709 non-null object
SHORT_TITLE    634709 non-null object
LONG_TITLE     634709 non-null object
dtypes: int64(3), object(3)
memory usage: 33.9+ MB


In [55]:
DataFrameSummary(diagnoses).summary()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
count,634709,634709,634709,NaN,NaN,NaN
mean,39354.7,150023,7.95513,NaN,NaN,NaN
std,29428.5,28873.9,6.10268,NaN,NaN,NaN
min,2,100001,1,NaN,NaN,NaN
25%,14813,125047,3,NaN,NaN,NaN
50%,28983,150151,6,NaN,NaN,NaN
75%,64287,174978,11,NaN,NaN,NaN
max,99999,199999,39,NaN,NaN,NaN
counts,634709,634709,634709,634709,634709,634709
uniques,46517,58925,39,6841,6770,6839


### Filter diagnosis on stays

In [56]:
def filter_diagnoses_on_stays(diagnoses, stays):
    return diagnoses.merge(stays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID']].drop_duplicates(), how='inner',
                           left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])

In [57]:
diagnoses = filter_diagnoses_on_stays(diagnoses, stays)

In [58]:
diagnoses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477958 entries, 0 to 477957
Data columns (total 7 columns):
SUBJECT_ID     477958 non-null int64
HADM_ID        477958 non-null int64
SEQ_NUM        477958 non-null int64
ICD9_CODE      477958 non-null object
SHORT_TITLE    477958 non-null object
LONG_TITLE     477958 non-null object
ICUSTAY_ID     477958 non-null int64
dtypes: int64(4), object(3)
memory usage: 29.2+ MB


In [59]:
DataFrameSummary(diagnoses).summary()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE,ICUSTAY_ID
count,477958,477958,477958,NaN,NaN,NaN,477958
mean,41179.3,150133,7.98156,NaN,NaN,NaN,250173
std,29647.2,28890.3,5.95499,NaN,NaN,NaN,28894.9
min,3,100001,1,NaN,NaN,NaN,200001
25%,16044,125039,3,NaN,NaN,NaN,225197
50%,30469,150409,7,NaN,NaN,NaN,250185
75%,66761,175171,11,NaN,NaN,NaN,275335
max,99999,199999,39,NaN,NaN,NaN,299999
counts,477958,477958,477958,477958,477958,477958,477958
uniques,33797,42269,39,6169,6102,6168,42269


### Save diagnoses df

In [60]:
diagnoses.to_csv(os.path.join(INTERIM_DATA, 'all_diagnoses.csv'), index=False)

In [61]:
#!ls $INTERIM_DATA

In [62]:
#!du -h $INTERIM_DATA

### Save diagnoses counts

In [63]:
def count_icd_codes(diagnoses, output_path=None):
    codes = diagnoses[['ICD9_CODE','SHORT_TITLE','LONG_TITLE']].drop_duplicates().set_index('ICD9_CODE')
    codes['COUNT'] = diagnoses.groupby('ICD9_CODE')['ICUSTAY_ID'].count()
    codes.COUNT = codes.COUNT.fillna(0).astype(int)
    codes = codes.loc[codes.COUNT>0]
    if output_path:
        codes.to_csv(output_path, index_label='ICD9_CODE')
    return codes.sort_values('COUNT', ascending=False).reset_index()

In [64]:
count_icd_codes(diagnoses, output_path=os.path.join(INTERIM_DATA, 'diagnosis_counts.csv'))

,ICD9_CODE,SHORT_TITLE,LONG_TITLE,COUNT
0,4019,Hypertension NOS,Unspecified essential hypertension,17343
1,4280,CHF NOS,"Congestive heart failure, unspecified",10601
2,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...,10345
3,42731,Atrial fibrillation,Atrial fibrillation,10313
4,25000,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...,7486
5,5849,Acute kidney failure NOS,"Acute kidney failure, unspecified",7343
6,2724,Hyperlipidemia NEC/NOS,Other and unspecified hyperlipidemia,7282
7,51881,Acute respiratry failure,Acute respiratory failure,5744
8,53081,Esophageal reflux,Esophageal reflux,5333
9,5990,Urin tract infection NOS,"Urinary tract infection, site not specified",5240


In [65]:
#!ls $INTERIM_DATA

## Phenotypes

**Add intro**

Get phenotype labels:

In [66]:
%pwd

'/home/alex/Dropbox/Jobb/projects/ML/medGPU1-alex/MIMIC3/nbs'

In [67]:
#!wget https://raw.githubusercontent.com/YerevaNN/mimic3-benchmarks/master/resources/hcup_ccs_2015_definitions.yaml

In [68]:
phenotype_definitions = f'{NB_DIR}/hcup_ccs_2015_definitions.yaml'

In [69]:
diagnosis_labels = [ '4019', '4280', '41401', '42731', '25000', '5849', '2724', '51881', '53081', '5990', '2720', '2859', '2449', 
                    '486', '2762', '2851', '496', 'V5861', '99592', '311', '0389', '5859', '5070', '40390', '3051', '412', 'V4581', 
                    '2761', '41071', '2875', '4240', 'V1582', 'V4582', 'V5867', '4241', '40391', '78552', '5119', '42789', '32723', 
                    '49390', '9971', '2767', '2760', '2749', '4168', '5180', '45829', '4589', '73300', '5845', '78039', '5856', '4271', 
                    '4254', '4111', 'V1251', '30000', '3572', '60000', '27800', '41400', '2768', '4439', '27651', 'V4501', '27652', 
                    '99811', '431', '28521', '2930', '7907', 'E8798', '5789', '79902', 'V4986', 'V103', '42832', 'E8788', '00845', 
                    '5715', '99591', '07054', '42833', '4275', '49121', 'V1046', '2948', '70703', '2809', '5712', '27801', '42732', 
                    '99812', '4139', '3004', '2639', '42822', '25060', 'V1254', '42823', '28529', 'E8782', '30500', '78791', '78551', 
                    'E8889', '78820', '34590', '2800', '99859', 'V667', 'E8497', '79092', '5723', '3485', '5601', '25040', '570', 
                    '71590', '2869', '2763', '5770', 'V5865', '99662', '28860', '36201', '56210' ]

In [70]:
def add_hcup_ccs_2015_groups(diagnoses, definitions):
    def_map = {}
    for dx in definitions:
        for code in definitions[dx]['codes']:
            def_map[code] = (dx, definitions[dx]['use_in_benchmark'])
    diagnoses['HCUP_CCS_2015'] = diagnoses.ICD9_CODE.apply(lambda c: def_map[c][0] if c in def_map else None)
    diagnoses['USE_IN_BENCHMARK'] = diagnoses.ICD9_CODE.apply(lambda c: int(def_map[c][1]) if c in def_map else None)
    return diagnoses

In [71]:
diagnoses.columns

Index(['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE', 'SHORT_TITLE',
       'LONG_TITLE', 'ICUSTAY_ID'],
      dtype='object')

In [72]:
phenotypes = add_hcup_ccs_2015_groups(diagnoses, yaml.load(open(phenotype_definitions, 'r')))

In [73]:
diagnoses.columns

Index(['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE', 'SHORT_TITLE',
       'LONG_TITLE', 'ICUSTAY_ID', 'HCUP_CCS_2015', 'USE_IN_BENCHMARK'],
      dtype='object')

In [74]:
def make_phenotype_label_matrix(phenotypes, stays=None):
    phenotypes = phenotypes[['ICUSTAY_ID', 'HCUP_CCS_2015']].loc[phenotypes.USE_IN_BENCHMARK > 0].drop_duplicates()
    phenotypes['VALUE'] = 1
    phenotypes = phenotypes.pivot(index='ICUSTAY_ID', columns='HCUP_CCS_2015', values='VALUE')
    if stays is not None:
        phenotypes = phenotypes.loc[stays.ICUSTAY_ID.sort_values()]
    return phenotypes.fillna(0).astype(int).sort_index(axis=0).sort_index(axis=1)

In [75]:
make_phenotype_label_matrix(phenotypes, stays).to_csv(os.path.join(INTERIM_DATA, 'phenotype_labels.csv'),
                                                      index=False, quoting=csv.QUOTE_NONNUMERIC)

/home/alex/anaconda2/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [76]:
#!ls $INTERIM_DATA

In [77]:
phenotype_labels = pd.read_csv(f'{INTERIM_DATA}/phenotype_labels.csv')

In [78]:
phenotype_labels.head()

,Acute and unspecified renal failure,Acute cerebrovascular disease,Acute myocardial infarction,Cardiac dysrhythmias,Chronic kidney disease,Chronic obstructive pulmonary disease and bronchiectasis,Complications of surgical procedures or medical care,Conduction disorders,Congestive heart failure; nonhypertensive,Coronary atherosclerosis and other heart disease,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,0,0,0,1,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,1,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Break up stays and diagnoses by subject

In [79]:
subjects = stays.SUBJECT_ID.unique()

In [80]:
subjects

array([  268,   269,   270,   272,   273,   274,   275,   276,   279,   281,   282,   283,   284,   285,
         286,   287,   290,   291,   292,   293, ..., 94897, 94903, 94906, 94908, 94911, 94912, 94915, 94916,
       94921, 94924, 94926, 94932, 94933, 94937, 94942, 94944, 94950, 94953, 94954, 94956])

In [81]:
len(subjects)

33798

In [82]:
def break_up_stays_by_subject(stays, output_path, subjects=None, verbose=1):
    subjects = stays.SUBJECT_ID.unique() if subjects is None else subjects
    nb_subjects = subjects.shape[0]
    for i, subject_id in enumerate(subjects):
        if verbose:
            sys.stdout.write('\rSUBJECT {0} of {1}...'.format(i+1, nb_subjects))
        dn = os.path.join(output_path, str(subject_id))
        try:
            os.makedirs(dn)
        except:
            pass

        stays.loc[stays.SUBJECT_ID == subject_id].sort_values(by='INTIME').to_csv(os.path.join(dn, 'stays.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [83]:
#!ls $INTERIM_DATA

In [84]:
break_up_stays_by_subject(stays, INTERIM_DATA, subjects=subjects, verbose=0)

In [85]:
def break_up_diagnoses_by_subject(diagnoses, output_path, subjects=None, verbose=1):
    subjects = diagnoses.SUBJECT_ID.unique() if subjects is None else subjects
    nb_subjects = subjects.shape[0]
    for i, subject_id in enumerate(subjects):
        if verbose:
            sys.stdout.write('\rSUBJECT {0} of {1}...'.format(i+1, nb_subjects))
        dn = os.path.join(output_path, str(subject_id))
        try:
            os.makedirs(dn)
        except:
            pass

        diagnoses.loc[diagnoses.SUBJECT_ID == subject_id].sort_values(by=['ICUSTAY_ID','SEQ_NUM']).to_csv(os.path.join(dn, 'diagnoses.csv'), index=False)
    if verbose:
        sys.stdout.write('DONE!\n')

In [86]:
break_up_diagnoses_by_subject(phenotypes, INTERIM_DATA, subjects=subjects, verbose=0)

**ADD EXPLANATION**

In [87]:
def read_events_table_by_row(mimic3_path, table, chartevents=330712484, labevents=27854056, outputevents=4349219):
    nb_rows = { 'chartevents': chartevents, 'labevents': labevents, 'outputevents': outputevents }
    reader = csv.DictReader(open(os.path.join(mimic3_path, table.upper() + '.csv'), 'r'))
    for i,row in enumerate(reader):
        if 'ICUSTAY_ID' not in row:
            row['ICUSTAY_ID'] = ''
        yield row, i, nb_rows[table.lower()]

In [88]:
def read_events_table_and_break_up_by_subject(mimic3_path, table, output_path, items_to_keep=None, subjects_to_keep=None, verbose=1):
    obs_header = [ 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM' ]
    if items_to_keep is not None:
        items_to_keep = set([ str(s) for s in items_to_keep ])
    if subjects_to_keep is not None:
        subjects_to_keep = set([ str(s) for s in subjects_to_keep ])

    class DataStats(object):
        def __init__(self):
            self.curr_subject_id = ''
            self.last_write_no = 0
            self.last_write_nb_rows = 0
            self.last_write_subject_id = ''
            self.curr_obs = []

    data_stats = DataStats()

    def write_current_observations():
        data_stats.last_write_no += 1
        data_stats.last_write_nb_rows = len(data_stats.curr_obs)
        data_stats.last_write_subject_id = data_stats.curr_subject_id
        dn = os.path.join(output_path, str(data_stats.curr_subject_id))
        try:
            os.makedirs(dn)
        except:
            pass
        fn = os.path.join(dn, 'events.csv')
        if not os.path.exists(fn) or not os.path.isfile(fn):
            f = open(fn, 'w')
            f.write(','.join(obs_header) + '\n')
            f.close()
        w = csv.DictWriter(open(fn, 'a'), fieldnames=obs_header, quoting=csv.QUOTE_MINIMAL)
        w.writerows(data_stats.curr_obs)
        data_stats.curr_obs = []
    
    for row, row_no, nb_rows in read_events_table_by_row(mimic3_path, table):
        if verbose and (row_no % 100000 == 0):
            if data_stats.last_write_no != '':
                sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...last write '
                                 '({3}) {4} rows for subject {5}'.format(table, row_no, nb_rows,
                                                                         data_stats.last_write_no,
                                                                         data_stats.last_write_nb_rows,
                                                                         data_stats.last_write_subject_id))
            else:
                sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...'.format(table, row_no, nb_rows))
        
        if (subjects_to_keep is not None and row['SUBJECT_ID'] not in subjects_to_keep):
            continue
        if (items_to_keep is not None and row['ITEMID'] not in items_to_keep):
            continue
        
        row_out = { 'SUBJECT_ID': row['SUBJECT_ID'],
                    'HADM_ID': row['HADM_ID'],
                    'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                    'CHARTTIME': row['CHARTTIME'],
                    'ITEMID': row['ITEMID'],
                    'VALUE': row['VALUE'],
                    'VALUEUOM': row['VALUEUOM'] }
        if data_stats.curr_subject_id != '' and data_stats.curr_subject_id != row['SUBJECT_ID']:
            write_current_observations()
        data_stats.curr_obs.append(row_out)
        data_stats.curr_subject_id = row['SUBJECT_ID']
        
    if data_stats.curr_subject_id != '':
        write_current_observations()

    if verbose:
        sys.stdout.write('\rfinished processing {0}: ROW {1} of {2}...last write '
                         '({3}) {4} rows for subject {5}...DONE!\n'.format(table, row_no, nb_rows,
                                                                 data_stats.last_write_no,
                                                                 data_stats.last_write_nb_rows,
                                                                 data_stats.last_write_subject_id))

In [89]:
itemids_file=None

In [90]:
items_to_keep = set(
    [int(itemid) for itemid in dataframe_from_csv(itemids_file)['ITEMID'].unique()]) if itemids_file else None

In [91]:
items_to_keep

In [92]:
event_tables = ['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS']

In [93]:
input("Are you sure you haven't done this already? Takes a while!")
for table in event_tables:
    read_events_table_and_break_up_by_subject(RAW_DATA, table, INTERIM_DATA, items_to_keep=items_to_keep,
                                              subjects_to_keep=subjects, verbose=1)

KeyboardInterrupt: 

**Checks**

In [94]:
stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD', 'AGE',
       'MORTALITY_INUNIT', 'MORTALITY', 'MORTALITY_INHOSPITAL'],
      dtype='object')

In [95]:
tmp_all_stays = pd.read_csv(f'{INTERIM_DATA}/all_stays.csv')

In [96]:
tmp_all_stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD', 'AGE',
       'MORTALITY_INUNIT', 'MORTALITY', 'MORTALITY_INHOSPITAL'],
      dtype='object')

In [97]:
tmp_stays = pd.read_csv(f'{INTERIM_DATA}/94942/stays.csv')

In [98]:
tmp_stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'DOD', 'AGE',
       'MORTALITY_INUNIT', 'MORTALITY', 'MORTALITY_INHOSPITAL'],
      dtype='object')

In [99]:
stays.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LAST_CAREUNIT,DBSOURCE,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,AGE,MORTALITY_INUNIT,MORTALITY,MORTALITY_INHOSPITAL
0,268,110404,280836,MICU,carevue,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,HISPANIC OR LATINO,DYSPNEA,F,2132-02-21,2198-02-18,66.030075,1,1,1
1,269,106296,206613,MICU,carevue,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,2170-11-05 11:04:00,2170-11-27 18:00:00,NaT,WHITE,SEPSIS;PILONIDAL ABSCESS,M,2130-09-30,NaT,40.127294,0,0,0
2,270,188028,220345,CCU,carevue,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939,2128-06-23 18:26:00,2128-06-27 12:31:00,NaT,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,M,2048-05-26,NaT,80.133229,0,0,0
3,272,164716,210407,CCU,carevue,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202,2186-12-25 21:06:00,2187-01-02 14:57:00,NaT,WHITE,PULMONARY EMBOLIS,M,2119-11-21,NaT,67.142139,0,0,0
4,273,158689,241507,MICU,carevue,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862,2141-04-19 06:11:00,2141-04-20 17:00:00,NaT,BLACK/AFRICAN AMERICAN,POLYSUBSTANCE OVERDOSE,M,2107-08-10,NaT,33.715776,0,0,0


In [100]:
tmp_stays

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LAST_CAREUNIT,DBSOURCE,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,AGE,MORTALITY_INUNIT,MORTALITY,MORTALITY_INHOSPITAL
0,94942,188423,264655,SICU,metavision,2179-12-25 19:07:29,2179-12-26 16:24:45,0.887,2179-12-25 19:06:00,2179-12-29 13:30:00,NaN,WHITE,CEREBRAL CONTUSION; S/P FALL,M,2099-10-31 00:00:00,NaN,80.204923,0,0,0


In [101]:
tmp_event = pd.read_csv(f'{INTERIM_DATA}/94942/events.csv')

In [102]:
tmp_event.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM
0,94942,188423,264655,2179-12-25 20:17:00,220045,99,bpm
1,94942,188423,264655,2179-12-25 20:17:00,220210,13,insp/min
2,94942,188423,264655,2179-12-25 20:17:00,220277,92,%
3,94942,188423,264655,2179-12-25 20:29:00,220179,134,mmHg
4,94942,188423,264655,2179-12-25 20:29:00,220180,61,mmHg


TODO: *Find all unique charttimes years*

# Validate events

**Explain**

In [103]:
n_events = 0                   # total number of events
empty_hadm = 0                 # HADM_ID is empty in events.csv. We exclude such events.
no_hadm_in_stay = 0            # HADM_ID does not appear in stays.csv. We exclude such events.
no_icustay = 0                 # ICUSTAY_ID is empty in events.csv. We try to fix such events.
recovered = 0                  # empty ICUSTAY_IDs are recovered according to stays.csv files (given HADM_ID)
could_not_recover = 0          # empty ICUSTAY_IDs that are not recovered. This should be zero.
icustay_missing_in_stays = 0   # ICUSTAY_ID does not appear in stays.csv. We exclude such events.

Get all subject folders:

In [104]:
def is_subject_folder(x):
    return str.isdigit(x)

In [105]:
subdirectories = os.listdir(INTERIM_DATA)
subjects = list(filter(is_subject_folder, subdirectories))

In [106]:
len(subdirectories), len(subjects)

(33802, 33798)

## Validate 

In [ ]:
for (index, subject) in enumerate(subjects):
        if index % 3000 == 0:
            print("processed {} / {} {}\r".format(index+1, len(subjects), ' '*10))

        stays_df = pd.read_csv(os.path.join(INTERIM_DATA, subject, 'stays.csv'), index_col=False,
                               dtype={'HADM_ID': str, "ICUSTAY_ID": str}, low_memory=False)
        stays_df.columns = stays_df.columns.str.upper()

        # assert that there is no row with empty ICUSTAY_ID or HADM_ID
        assert(not stays_df['ICUSTAY_ID'].isnull().any())
        assert(not stays_df['HADM_ID'].isnull().any())

        # assert there are no repetitions of ICUSTAY_ID or HADM_ID
        # since admissions with multiple ICU stays were excluded
        assert(len(stays_df['ICUSTAY_ID'].unique()) == len(stays_df['ICUSTAY_ID']))
        assert(len(stays_df['HADM_ID'].unique()) == len(stays_df['HADM_ID']))

        events_df = pd.read_csv(os.path.join(INTERIM_DATA, subject, 'events.csv'), index_col=False,
                                dtype={'HADM_ID': str, "ICUSTAY_ID": str}, low_memory=False)
        events_df.columns = events_df.columns.str.upper()
        n_events += events_df.shape[0]

        # we drop all events for them HADM_ID is empty
        # TODO: maybe we can recover HADM_ID by looking at ICUSTAY_ID
        empty_hadm += events_df['HADM_ID'].isnull().sum()
        events_df = events_df.dropna(subset=['HADM_ID'])

        merged_df = events_df.merge(stays_df, left_on=['HADM_ID'], right_on=['HADM_ID'],
                                    how='left', suffixes=['', '_r'], indicator=True)

        # we drop all events for which HADM_ID is not listed in stays.csv
        # since there is no way to know the targets of that stay (for example mortality)
        no_hadm_in_stay += (merged_df['_merge'] == 'left_only').sum()
        merged_df = merged_df[merged_df['_merge'] == 'both']

        # if ICUSTAY_ID is empty in stays.csv, we try to recover it
        # we exclude all events for which we could not recover ICUSTAY_ID
        cur_no_icustay = merged_df['ICUSTAY_ID'].isnull().sum()
        no_icustay += cur_no_icustay
        merged_df.loc[:, 'ICUSTAY_ID'] = merged_df['ICUSTAY_ID'].fillna(merged_df['ICUSTAY_ID_r'])
        recovered += cur_no_icustay - merged_df['ICUSTAY_ID'].isnull().sum()
        could_not_recover += merged_df['ICUSTAY_ID'].isnull().sum()
        merged_df = merged_df.dropna(subset=['ICUSTAY_ID'])

        # now we take a look at the case when ICUSTAY_ID is present in events.csv, but not in stays.csv
        # this mean that ICUSTAY_ID in events.csv is not the same as that of stays.csv for the same HADM_ID
        # we drop all such events
        icustay_missing_in_stays += (merged_df['ICUSTAY_ID'] != merged_df['ICUSTAY_ID_r']).sum()
        merged_df = merged_df[(merged_df['ICUSTAY_ID'] == merged_df['ICUSTAY_ID_r'])]
        
        to_write = merged_df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM']]
        to_write.to_csv(os.path.join(INTERIM_DATA, subject, 'events.csv'), index=False)

In [ ]:
assert(could_not_recover == 0)
print('n_events: {}'.format(n_events))
print('empty_hadm: {}'.format(empty_hadm))
print('no_hadm_in_stay: {}'.format(no_hadm_in_stay))
print('no_icustay: {}'.format(no_icustay))
print('recovered: {}'.format(recovered))
print('could_not_recover: {}'.format(could_not_recover))
print('icustay_missing_in_stays: {}'.format(icustay_missing_in_stays))

### Write to df

# Extract episodes from subjects

In [107]:
variable_map_file = f'{MIMIC3_BENCHMARK_LOCATION}/resources/itemid_to_variable_map.csv'
reference_range_file = f'{MIMIC3_BENCHMARK_LOCATION}/resources/variable_ranges.csv'

In [108]:
def read_itemid_to_variable_map(fn, variable_column='LEVEL2'):
    var_map = dataframe_from_csv(fn, index_col=None).fillna('').astype(str)
    #var_map[variable_column] = var_map[variable_column].apply(lambda s: s.lower())
    var_map.COUNT = var_map.COUNT.astype(int)
    var_map = var_map.loc[(var_map[variable_column] != '') & (var_map.COUNT>0)]
    var_map = var_map.loc[(var_map.STATUS == 'ready')]
    var_map.ITEMID = var_map.ITEMID.astype(int)
    var_map = var_map[[variable_column, 'ITEMID', 'MIMIC LABEL']].set_index('ITEMID')
    return var_map.rename({variable_column: 'VARIABLE', 'MIMIC LABEL': 'MIMIC_LABEL'}, axis=1)

In [109]:
var_map = read_itemid_to_variable_map(variable_map_file)

In [110]:
variables = var_map.VARIABLE.unique()

In [111]:
variables

array(['Capillary refill rate', 'Diastolic blood pressure', 'Fraction inspired oxygen',
       'Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total',
       'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure',
       'Oxygen saturation', 'pH', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight'],
      dtype=object)

In [112]:
def read_stays(subject_path):
    stays = pd.read_csv(os.path.join(subject_path, 'stays.csv'), header=0, index_col=None)
    stays.INTIME = pd.to_datetime(stays.INTIME)
    stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
    stays.DOB = pd.to_datetime(stays.DOB)
    stays.DOD = pd.to_datetime(stays.DOD)
    stays.DEATHTIME = pd.to_datetime(stays.DEATHTIME)
    stays.sort_values(by=['INTIME', 'OUTTIME'], inplace=True)
    return stays

def read_diagnoses(subject_path):
    return pd.read_csv(os.path.join(subject_path, 'diagnoses.csv'), header=0, index_col=None)

def read_events(subject_path, remove_null=True):
    events = pd.read_csv(os.path.join(subject_path, 'events.csv'), header=0, index_col=None)
    if remove_null:
        events = events.loc[events.VALUE.notnull()]
    events.CHARTTIME = pd.to_datetime(events.CHARTTIME)
    events.HADM_ID = events.HADM_ID.fillna(value=-1).astype(int)
    events.ICUSTAY_ID = events.ICUSTAY_ID.fillna(value=-1).astype(int)
    events.VALUEUOM = events.VALUEUOM.fillna('').astype(str)
#    events.sort_values(by=['CHARTTIME', 'ITEMID', 'ICUSTAY_ID'], inplace=True)
    return events

In [113]:
def assemble_episodic_data(stays, diagnoses):
    data = { 'Icustay': stays.ICUSTAY_ID, 'Age': stays.AGE, 'Length of Stay': stays.LOS,
                    'Mortality': stays.MORTALITY }
    data.update(transform_gender(stays.GENDER))
    data.update(transform_ethnicity(stays.ETHNICITY))
    data['Height'] = np.nan
    data['Weight'] = np.nan
    data = DataFrame(data).set_index('Icustay')
    data = data[['Ethnicity', 'Gender', 'Age', 'Height', 'Weight', 'Length of Stay', 'Mortality']]
    return data.merge(extract_diagnosis_labels(diagnoses), left_index=True, right_index=True)

In [114]:
g_map = { 'F': 1, 'M': 2, 'OTHER': 3, '': 0 }
def transform_gender(gender_series):
    global g_map
    return { 'Gender': gender_series.fillna('').apply(lambda s: g_map[s] if s in g_map else g_map['OTHER']) }

e_map = { 'ASIAN': 1,
          'BLACK': 2,
          'HISPANIC': 3,
          'WHITE': 4,
          'OTHER': 5, # map everything else to 5 (OTHER)
          'UNABLE TO OBTAIN': 0,
          'PATIENT DECLINED TO ANSWER': 0,
          'UNKNOWN': 0,
          '': 0 }
def transform_ethnicity(ethnicity_series):
    global e_map
    
    def aggregate_ethnicity(ethnicity_str):
        return ethnicity_str.replace(' OR ', '/').split(' - ')[0].split('/')[0]

    ethnicity_series = ethnicity_series.apply(aggregate_ethnicity)
    return { 'Ethnicity': ethnicity_series.fillna('').apply(lambda s: e_map[s] if s in e_map else e_map['OTHER']) }



diagnosis_labels = [ '4019', '4280', '41401', '42731', '25000', '5849', '2724', '51881', '53081', '5990', '2720', '2859', '2449', '486', '2762', '2851', '496', 'V5861', '99592', '311', '0389', '5859', '5070', '40390', '3051', '412', 'V4581', '2761', '41071', '2875', '4240', 'V1582', 'V4582', 'V5867', '4241', '40391', '78552', '5119', '42789', '32723', '49390', '9971', '2767', '2760', '2749', '4168', '5180', '45829', '4589', '73300', '5845', '78039', '5856', '4271', '4254', '4111', 'V1251', '30000', '3572', '60000', '27800', '41400', '2768', '4439', '27651', 'V4501', '27652', '99811', '431', '28521', '2930', '7907', 'E8798', '5789', '79902', 'V4986', 'V103', '42832', 'E8788', '00845', '5715', '99591', '07054', '42833', '4275', '49121', 'V1046', '2948', '70703', '2809', '5712', '27801', '42732', '99812', '4139', '3004', '2639', '42822', '25060', 'V1254', '42823', '28529', 'E8782', '30500', '78791', '78551', 'E8889', '78820', '34590', '2800', '99859', 'V667', 'E8497', '79092', '5723', '3485', '5601', '25040', '570', '71590', '2869', '2763', '5770', 'V5865', '99662', '28860', '36201', '56210' ]
def extract_diagnosis_labels(diagnoses):
    global diagnosis_labels
    diagnoses['VALUE'] = 1
    labels = diagnoses[['ICUSTAY_ID', 'ICD9_CODE', 'VALUE']].drop_duplicates().pivot(index='ICUSTAY_ID', columns='ICD9_CODE', values='VALUE').fillna(0).astype(int)    
    for l in diagnosis_labels:
        if l not in labels:
            labels[l] = 0
    labels = labels[diagnosis_labels]
    return labels.rename(dict(zip(diagnosis_labels, [ 'Diagnosis ' + d for d in diagnosis_labels])), axis=1)


def map_itemids_to_variables(events, var_map):
    return events.merge(var_map, left_on='ITEMID', right_index=True)

In [115]:
# SBP: some are strings of type SBP/DBP
def clean_sbp(df):
    v = df.VALUE.astype(str)
    idx = v.apply(lambda s: '/' in s)
    v.loc[idx] = v[idx].apply(lambda s: re.match('^(\d+)/(\d+)$', s).group(1))
    return v.astype(float)

def clean_dbp(df):
    v = df.VALUE.astype(str)
    idx = v.apply(lambda s: '/' in s)
    v.loc[idx] = v[idx].apply(lambda s: re.match('^(\d+)/(\d+)$', s).group(2))
    return v.astype(float)

# CRR: strings with brisk, <3 normal, delayed, or >3 abnormal
def clean_crr(df):
    v = Series(np.zeros(df.shape[0]), index=df.index)
    v[:] = np.nan
    
    # when df.VALUE is empty, dtype can be float and comparision with string
    # raises an exception, to fix this we change dtype to str
    df.VALUE = df.VALUE.astype(str)
    
    v.loc[(df.VALUE == 'Normal <3 secs') | (df.VALUE == 'Brisk')] = 0
    v.loc[(df.VALUE == 'Abnormal >3 secs') | (df.VALUE == 'Delayed')] = 1
    return v

# FIO2: many 0s, some 0<x<0.2 or 1<x<20
def clean_fio2(df):
    v = df.VALUE.astype(float)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'torr' not in s.lower()) & (v>1.0)
    #idx = df.VALUEUOM.fillna('').apply(lambda s: 'torr' not in s.lower()) & (df.VALUE>1.0)
    v.loc[idx] = v[idx] / 100.
    return v

# GLUCOSE, PH: sometimes have ERROR as value
def clean_lab(df):
    v = df.VALUE
    idx = v.apply(lambda s: type(s) is str and not re.match('^(\d+(\.\d*)?|\.\d+)$', s))
    v.loc[idx] = np.nan
    return v.astype(float)

# O2SAT: small number of 0<x<=1 that should be mapped to 0-100 scale
def clean_o2sat(df):
    # change "ERROR" to NaN
    v = df.VALUE
    idx = v.apply(lambda s: type(s) is str and not re.match('^(\d+(\.\d*)?|\.\d+)$', s))
    v.loc[idx] = np.nan
    
    v = v.astype(float)
    idx = (v<=1)
    v.loc[idx] = v[idx] * 100.
    return v

# Temperature: map Farenheit to Celsius, some ambiguous 50<x<80
def clean_temperature(df):
    v = df.VALUE.astype(float)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'F' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'F' in s.lower()) | (v >= 79)
    v.loc[idx] = (v[idx] - 32) * 5. / 9
    return v

# Weight: some really light/heavy adults: <50 lb, >450 lb, ambiguous oz/lb
# Children are tough for height, weight
def clean_weight(df):
    v = df.VALUE.astype(float)
    # ounces
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'oz' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'oz' in s.lower())
    v.loc[idx] = v[idx] / 16.
    # pounds
    idx = idx | df.VALUEUOM.fillna('').apply(lambda s: 'lb' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'lb' in s.lower())
    v.loc[idx] = v[idx] * 0.453592
    return v

# Height: some really short/tall adults: <2 ft, >7 ft)
# Children are tough for height, weight
def clean_height(df):
    v = df.VALUE.astype(float)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'in' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'in' in s.lower())
    v.loc[idx] = np.round(v[idx] * 2.54)
    return v

# ETCO2: haven't found yet
# Urine output: ambiguous units (raw ccs, ccs/kg/hr, 24-hr, etc.)
# Tidal volume: tried to substitute for ETCO2 but units are ambiguous
# Glascow coma scale eye opening
# Glascow coma scale motor response
# Glascow coma scale total
# Glascow coma scale verbal response
# Heart Rate
# Respiratory rate
# Mean blood pressure
clean_fns = {
    'Capillary refill rate': clean_crr,
    'Diastolic blood pressure': clean_dbp,
    'Systolic blood pressure': clean_sbp,
    'Fraction inspired oxygen': clean_fio2,
    'Oxygen saturation': clean_o2sat,
    'Glucose': clean_lab,
    'pH': clean_lab,
    'Temperature': clean_temperature,
    'Weight': clean_weight,
    'Height': clean_height
}

def clean_events(events):
    global cleaning_fns
    for var_name, clean_fn in clean_fns.items():
        idx = (events.VARIABLE == var_name)
        try:
            events.loc[idx,'VALUE'] = clean_fn(events.loc[idx])
        except Exception as e:
            print("Exception in clean_events:", clean_fn.__name__, e)
            print("number of rows:", np.sum(idx))
            print("values:", events.loc[idx])
            exit()
    return events.loc[events.VALUE.notnull()]

In [116]:
def convert_events_to_timeseries(events, variable_column='VARIABLE', variables=[]):
    metadata = events[['CHARTTIME', 'ICUSTAY_ID']].sort_values(by=['CHARTTIME', 'ICUSTAY_ID'])\
                    .drop_duplicates(keep='first').set_index('CHARTTIME')
    timeseries = events[['CHARTTIME', variable_column, 'VALUE']]\
                    .sort_values(by=['CHARTTIME', variable_column, 'VALUE'], axis=0)\
                    .drop_duplicates(subset=['CHARTTIME', variable_column], keep='last')
    timeseries = timeseries.pivot(index='CHARTTIME', columns=variable_column, values='VALUE').merge(metadata, left_index=True, right_index=True)\
                    .sort_index(axis=0).reset_index()
    for v in variables:
        if v not in timeseries:
            timeseries[v] = np.nan
    return timeseries

In [117]:
def get_events_for_stay(events, icustayid, intime=None, outtime=None):
    idx = (events.ICUSTAY_ID == icustayid)
    if intime is not None and outtime is not None:
        idx = idx | ((events.CHARTTIME >= intime) & (events.CHARTTIME <= outtime))
    events = events.loc[idx]
    del events['ICUSTAY_ID']
    return events

In [118]:
def add_hours_elpased_to_events(events, dt, remove_charttime=True):
    events['HOURS'] = (events.CHARTTIME - dt).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60
    if remove_charttime:
    	del events['CHARTTIME']
    return events

In [119]:
def get_first_valid_from_timeseries(timeseries, variable):
	if variable in timeseries:
		idx = timeseries[variable].notnull()
		if idx.any():
			loc = np.where(idx)[0][0]
			return timeseries[variable].iloc[loc]
	return np.nan

In [120]:
verbose=0

for subject_dir in os.listdir(INTERIM_DATA):
    dn = os.path.join(INTERIM_DATA, subject_dir)
    try:
        subject_id = int(subject_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        continue
    if verbose:
        sys.stdout.write('Subject {}: '.format(subject_id))
        sys.stdout.flush()

    try:
        if verbose: 
            sys.stdout.write('reading...')
            sys.stdout.flush()
        stays = read_stays(os.path.join(INTERIM_DATA, subject_dir))
        diagnoses = read_diagnoses(os.path.join(INTERIM_DATA, subject_dir))
        events = read_events(os.path.join(INTERIM_DATA, subject_dir))
    except:
        sys.stdout.write('error reading from disk!\n')
        continue
    else:
        if verbose:
            sys.stdout.write('got {0} stays, {1} diagnoses, {2} events...'.format(stays.shape[0], diagnoses.shape[0], events.shape[0]))
            sys.stdout.flush()

    episodic_data = assemble_episodic_data(stays, diagnoses)

    if verbose: 
        sys.stdout.write('cleaning and converting to time series...')
        sys.stdout.flush()
    events = map_itemids_to_variables(events, var_map)
    events = clean_events(events)
    if events.shape[0] == 0:
        if verbose: sys.stdout.write('no valid events!\n')
        continue
    timeseries = convert_events_to_timeseries(events, variables=variables)
    
    if verbose: 
        sys.stdout.write('extracting separate episodes...')
        sys.stdout.flush()

    for i in range(stays.shape[0]):
        stay_id = stays.ICUSTAY_ID.iloc[i]
        if verbose: 
            sys.stdout.write(' {}'.format(stay_id))
            sys.stdout.flush()
        intime = stays.INTIME.iloc[i]
        outtime = stays.OUTTIME.iloc[i]

        episode = get_events_for_stay(timeseries, stay_id, intime, outtime)
        if episode.shape[0] == 0:
            if verbose: 
                sys.stdout.write(' (no data!)')
                sys.stdout.flush()
            continue

        episode = add_hours_elpased_to_events(episode, intime).set_index('HOURS').sort_index(axis=0)
        episodic_data.Weight.loc[stay_id] = get_first_valid_from_timeseries(episode, 'Weight')
        episodic_data.Height.loc[stay_id] = get_first_valid_from_timeseries(episode, 'Height')
        episodic_data.loc[episodic_data.index==stay_id].to_csv(os.path.join(INTERIM_DATA, subject_dir, 'episode{}.csv'.format(i+1)), index_label='Icustay')
        columns = list(episode.columns)
        columns_sorted = sorted(columns, key=(lambda x: "" if x == "Hours" else x))
        episode = episode[columns_sorted]
        episode.to_csv(os.path.join(INTERIM_DATA, subject_dir, 'episode{}_timeseries.csv'.format(i+1)), index_label='Hours')
    if verbose: sys.stdout.write(' DONE!\n')

/home/alex/anaconda2/envs/fastai/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/alex/anaconda2/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alex/anaconda2/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alex/anaconda2/envs/fastai/lib/python3.6/si